In [2]:
import os
from dotenv import load_dotenv

load_dotenv()


True

In [3]:
from langchain_teddynote.tools.tavily import TavilySearch
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from typing import Annotated, TypedDict
import ast
import re

In [7]:
import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import os

# Step 1: Load CSV data
file_path = "../data/media.csv"  # This should be the uploaded CSV file
df = pd.read_csv(file_path, header=None)

# Step 2: Define column names manually since no header in the file
df.columns = [
    "media_id", "media_name", "location", "size", "duration", "media_type",
    "operating_hours", "is_digital", "slot_count", "is_available", "unit_price",
    "location_description", "image_day", "image_night", "image_map",
    "population_target", "media_characteristics", "case_examples"
]

# Step 3: Prepare embedding model using HuggingFace (MiniLM)
class BERTSentenceEmbedding:
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def embed_documents(self, texts):
        return [self._embed(text) for text in texts]

    def embed_query(self, text):
        return self._embed(text)

    def _embed(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze(0)
        return cls_embedding.cpu().numpy()

embedding_function = BERTSentenceEmbedding()

# Step 4: Construct documents for Chroma
def build_text(row):
    return f"""
    위치 설명: {row['location_description']}
    타겟: {row['population_target']}
    매체 특징: {row['media_characteristics']}
    집행 사례: {row['case_examples']}
    """

docs = []
for i, row in df.iterrows():
    doc = Document(
        page_content=build_text(row),
        metadata={
            "media_id": str(row["media_id"]),
            "media_name": row["media_name"],
            "location": row["location"],
            "media_type": row["media_type"],
            "population_target": row["population_target"],
            "media_characteristics": row["media_characteristics"],
            "case_examples": row["case_examples"]
        }
    )
    docs.append(doc)

# Step 5: Store in Chroma
chroma_collection = Chroma.from_documents(
    documents=docs,
    embedding=embedding_function,
    collection_name="media",
    persist_directory="./chroma_media"
)


In [19]:
chroma_collection.persist()

/var/folders/x8/x9mhzs_j0yx8btgk3t9m0vvc0000gn/T/ipykernel_2183/570867232.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  chroma_collection.persist()


In [20]:
print("저장된 문서 수:", chroma_collection._collection.count())  # 또는 chroma_collection._collection.count()


저장된 문서 수: 51


In [11]:
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma

# (전제) 임베딩 함수 정의되어 있음
embedding_function = BERTSentenceEmbedding()

# ✅ LLM 및 벡터DB 연결
llm = ChatOpenAI(model="gpt-4o-mini")
chroma_collection = Chroma(
    collection_name="media",
    embedding_function=embedding_function,
    persist_directory="./chroma_media"
)

# ✅ 출력 구조 정의
class MatchingAgent(TypedDict):
    brand: dict
    top_3_matches: list[dict]
    matching_reason: str
    sales_talking_points: list[str]

# ✅ 에이전트 정의
def media_matcher_agent(brand_name: str, recent_issue: str, brand_description: str) -> MatchingAgent:
    query_text = f"{recent_issue} / {brand_description}"
    results = chroma_collection.similarity_search_with_score(query_text, k=10)  # 넉넉하게 검색

    # 중복 제거된 매체 3개만 추출
    seen_ids = set()
    top_matches = []
    for doc, _ in results:
        meta = doc.metadata
        if meta["media_id"] in seen_ids:
            continue
        seen_ids.add(meta["media_id"])
        reason = f"{meta['population_target']}을 타겟으로 하며, '{meta['media_characteristics']}' 특성을 가짐. '{meta['case_examples']}' 등 유사 캠페인 존재."
        top_matches.append({
            "rank": len(top_matches) + 1,
            "media_id": meta["media_id"],
            "media_name": meta["media_name"],
            "location": meta["location"],
            "media_type": meta["media_type"],
            "individual_reason": reason
        })
        if len(top_matches) == 3:
            break

    # ✅ 매칭 이유 생성 프롬프트
    match_prompt = f"""
    브랜드명: {brand_name}
    최근 마케팅 이슈: {recent_issue}
    브랜드 설명: {brand_description}
    추천된 매체:
    1. {top_matches[0]['media_name']} ({top_matches[0]['location']}) - {top_matches[0]['individual_reason']}
    2. {top_matches[1]['media_name']} ({top_matches[1]['location']}) - {top_matches[1]['individual_reason']}
    3. {top_matches[2]['media_name']} ({top_matches[2]['location']}) - {top_matches[2]['individual_reason']}

    위 정보를 바탕으로, 이 3개 매체가 왜 이 브랜드에 적합한지 요약해줘.
    포멀하고 간결한 문장으로 1~2줄로 작성해줘.
    """
    matching_reason = llm.invoke(match_prompt).content.strip()

    # ✅ 세일즈 포인트 생성 프롬프트
    sales_prompt = f"""
    브랜드: {brand_name}
    이슈: {recent_issue}
    브랜드 설명: {brand_description}
    추천 매체: {', '.join([m['media_name'] for m in top_matches])}

    광고주에게 제안할 세일즈 문장을 3줄로 작성해줘.
    - 1줄: 이슈 기반 축하 + 제안 개요
    - 2줄: 추천 매체를 활용한 전략 제안
    - 3줄: 브랜드 효과 강조
    모든 문장은 B2B 세일즈 스타일로 포멀하게 써줘.
    """
    sales_points = llm.invoke(sales_prompt).content.strip().split("\n")
    sales_points = [line.strip("- ").strip() for line in sales_points if line.strip()][:3]

    # ✅ 최종 반환
    return {
        "brand": {
            "name": brand_name,
            "recent_issue": recent_issue,
            "target_audience": brand_description
        },
        "top_3_matches": top_matches,
        "matching_reason": matching_reason,
        "sales_talking_points": sales_points
    }


In [8]:
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma

# (전제) 임베딩 함수 정의되어 있음
embedding_function = BERTSentenceEmbedding()

# ✅ LLM 및 벡터DB 연결
llm = ChatOpenAI(model="gpt-4o-mini")
chroma_collection = Chroma(
    collection_name="media",
    embedding_function=embedding_function,
    persist_directory="./chroma_media"
)

# ✅ 출력 구조 정의 (수정)
class MatchingAgent(TypedDict):
    brand: dict
    top_3_matches: list[dict]
    sales_call_script: str  # 변경된 부분

# ✅ 에이전트 정의 (수정)
def media_matcher_agent(brand_name: str, recent_issue: str, brand_description: str) -> MatchingAgent:
    query_text = f"{recent_issue} / {brand_description}"
    results = chroma_collection.similarity_search_with_score(query_text, k=10)  # 넉넉하게 검색

    # 중복 제거된 매체 3개만 추출
    seen_ids = set()
    top_matches = []
    for doc, _ in results:
        meta = doc.metadata
        if meta["media_id"] in seen_ids:
            continue
        seen_ids.add(meta["media_id"])
        reason = f"{meta['population_target']}을 타겟으로 하며, '{meta['media_characteristics']}' 특성을 가짐. '{meta['case_examples']}' 등 유사 캠페인 존재."
        top_matches.append({
            "rank": len(top_matches) + 1,
            "media_id": meta["media_id"],
            "media_name": meta["media_name"],
            "location": meta["location"],
            "media_type": meta["media_type"],
            "individual_reason": reason
        })
        if len(top_matches) == 3:
            break

    # ✅ 1순위 매체 기반 영업 전화 스크립트 생성 (수정)
    script_prompt = f"""
    브랜드명: {brand_name}
    최근 마케팅 이슈: {recent_issue}
    브랜드 설명: {brand_description}
    추천 매체: {top_matches[0]['media_name']} ({top_matches[0]['location']}) - {top_matches[0]['individual_reason']}

    위 정보를 바탕으로, {brand_name}의 담당자에게 전화할 때 사용할 영업 스크립트를 3-5줄로 작성해주세요.
    스크립트는 다음을 포함해야 합니다:
    - 인사 및 자기소개
    - 브랜드의 최근 이슈 언급
    - 추천 매체({top_matches[0]['media_name']})가 왜 적합한지 설명
    - 미팅 제안
    
    실제 전화통화처럼 자연스럽게 작성해주세요.
    """
    sales_call_script = llm.invoke(script_prompt).content.strip()

    # ✅ 최종 반환 (수정)
    return {
        "brand": {
            "name": brand_name,
            "recent_issue": recent_issue,
            "target_audience": brand_description
        },
        "top_3_matches": top_matches,
        "sales_call_script": sales_call_script  # 변경된 부분
    }

/var/folders/x8/x9mhzs_j0yx8btgk3t9m0vvc0000gn/T/ipykernel_31585/2628144890.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_collection = Chroma(


In [21]:
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma

# (전제) 임베딩 함수 정의되어 있음
embedding_function = BERTSentenceEmbedding()

# ✅ LLM 및 벡터DB 연결
llm = ChatOpenAI(model="gpt-4o-mini")
chroma_collection = Chroma(
    collection_name="media",
    embedding_function=embedding_function,
    persist_directory="./chroma_media"
)

# ✅ 출력 구조 정의 (수정)
class MatchingAgent(TypedDict):
    brand: dict
    top_match: dict  # 변경된 부분: top_3_matches → top_match
    sales_call_script: str

# ✅ 에이전트 정의 (수정)
def media_matcher_agent(brand_name: str, recent_issue: str, brand_description: str) -> MatchingAgent:
    query_text = f"{recent_issue} / {brand_description}"
    results = chroma_collection.similarity_search_with_score(query_text, k=10)  # 넉넉하게 검색

    # 첫 번째 매체만 추출 (수정)
    top_match = None
    for doc, _ in results:
        meta = doc.metadata
        reason = f"{meta['population_target']}을 타겟으로 하며, '{meta['media_characteristics']}' 특성을 가짐. '{meta['case_examples']}' 등 유사 캠페인 존재."
        top_match = {
            "media_id": meta["media_id"],
            "media_name": meta["media_name"],
            "location": meta["location"],
            "media_type": meta["media_type"],
            "match_reason": reason
        }
        break  # 첫 번째 결과만 사용

    # ✅ 1순위 매체 기반 영업 전화 스크립트 생성
    script_prompt = f"""
    브랜드명: {brand_name}
    최근 마케팅 이슈: {recent_issue}
    브랜드 설명: {brand_description}
    추천 매체: {top_match['media_name']} ({top_match['location']}) - {top_match['match_reason']}

    위 정보를 바탕으로, {brand_name}의 담당자에게 전화할 때 사용할 영업 스크립트를 3-5줄로 작성해주세요.
    스크립트는 다음을 포함해야 합니다:
    - 인사 및 자기소개
    - 브랜드의 최근 이슈 언급
    - 추천 매체({top_match['media_name']})가 왜 적합한지 설명
    - 미팅 제안
    
    실제 영업 전화 통화처럼 정중하지만 자연스럽게 작성해주세요.
    """
    sales_call_script = llm.invoke(script_prompt).content.strip()

    # ✅ 최종 반환 (수정)
    return {
        "brand": {
            "name": brand_name,
            "recent_issue": recent_issue,
            "target_audience": brand_description
        },
        "top_match": top_match,  # 변경된 부분: top_3_matches → top_match
        "sales_call_script": sales_call_script
    }

    # return {
    #     "top_match": top_match,  
    #     "sales_call_script": sales_call_script
    # }

In [22]:
result = media_matcher_agent(
    brand_name="더바넷",
    recent_issue="2025년 3월 9일: 서울 잠실 롯데월드몰에 국내 첫 팝업스토어 오픈",
    brand_description="2021년 론칭한 캐주얼 브랜드로, 20·30세대 고객에게 가장 트렌디한 브랜드로 손꼽히며, 가방과 모자, 액세서리를 포함한 다양한 상품을 선보인다."
)

result

{'brand': {'name': '더바넷',
  'recent_issue': '2025년 3월 9일: 서울 잠실 롯데월드몰에 국내 첫 팝업스토어 오픈',
  'target_audience': '2021년 론칭한 캐주얼 브랜드로, 20·30세대 고객에게 가장 트렌디한 브랜드로 손꼽히며, 가방과 모자, 액세서리를 포함한 다양한 상품을 선보인다.'},
 'top_match': {'media_id': '17',
  'media_name': '가로변 버스쉘터 강남대로',
  'location': '서울시 강남구 강남대로 일대',
  'media_type': '버스정류장 쉘터 광고',
  'match_reason': "버스 이용객, 보행자, 전 연령층, 특히 출퇴근 시간대 직장인을 타겟으로 하며, '서울 전 지역 2100여 기가 설치된 생활밀착형 매체로, 지역 타겟팅 용이' 특성을 가짐. '음료 브랜드 시즌 캠페인(2023년 여름), 배달앱 신규 서비스(2024년 초)' 등 유사 캠페인 존재."},
 'sales_call_script': '안녕하세요, 더바넷 마케팅 팀의 [당신의 이름]입니다. 최근 서울 잠실 롯데월드몰에 국내 첫 팝업스토어 오픈 소식을 들었습니다. 축하드립니다! 귀사의 트렌디한 이미지를 강남대로의 가로변 버스쉘터에 광고하시면, 출퇴근 시간대 직장인들을 효과적으로 타겟팅할 수 있을 것입니다. 이 매체는 서울 전역에 2100여 기가 설치되어 있어 지역 타겟팅도 용이합니다. 이와 관련해 좀 더 자세한 이야기를 나누기 위해 미팅을 제안드리고 싶습니다. 가능하신 시간을 알려주시면 감사하겠습니다.'}